# AU-Ggregates AI Server — Kaggle Deployment

**Pipeline:** Phi-3-mini-4k-instruct (4-bit) > T5-text-to-SQL > Supabase

## Before you start
1. **Settings** (right panel) > **Accelerator** > **GPU T4 x2**
2. **Settings** > **Internet** > make sure it's **ON**
3. **Add-ons** > **Secrets** (or click the 🔑 key icon in right panel) > add these 4 secrets:
   - `SUPABASE_URL` — your Supabase project URL (e.g. `https://xxx.supabase.co`)
   - `SUPABASE_KEY` — your Supabase anon key (starts with `eyJ...`)
   - `HF_TOKEN` — HuggingFace token (starts with `hf_...`)
   - `NGROK_TOKEN` — ngrok auth token
4. **IMPORTANT**: Toggle each secret's switch to **Attached** (ON) for this notebook
5. After adding secrets, do **Run > Restart & Clear All Outputs** then re-run

## Pipeline
| Cell | What it does | Time |
|------|-------------|------|
| 1 | Install dependencies | ~2 min |
| 2 | Set secrets/keys | instant |
| 3 | Clone GitHub repo | ~10 sec |
| 4 | Verify GPU | instant |
| 5 | Login to HuggingFace | instant |
| 6 | Start server + ngrok tunnel | ~2 min |
| 6B | Monitor download progress | live |
| 7 | Test AI query | ~30 sec |
| 7B | Test query gating | ~30 sec |
| 8 | Health check + debug | instant |

In [ ]:
# ============================================================
# CELL 1: Install all dependencies
# ============================================================
# CRITICAL: Pin to single GPU before any torch import
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Core web framework
!pip install -q fastapi uvicorn[standard] pydantic python-multipart

# Database
!pip install -q "supabase>=2.0.0,<2.28.0" psycopg2-binary

# AI Models (Phi-3 + T5)
!pip install -q "transformers>=4.43.0,<4.48.0" "accelerate>=0.33.0" "bitsandbytes>=0.43.0" sentencepiece
!pip install -q sentence-transformers dateparser rapidfuzz

# HuggingFace login
!pip install -q huggingface_hub

# Utilities
!pip install -q python-dotenv requests loguru sentry-sdk python-json-logger
!pip install -q sqlparse apscheduler

# ngrok for public URL
!pip install -q pyngrok

# Jupyter async fix
!pip install -q nest_asyncio

print('\nAll dependencies installed!')

In [ ]:
# ============================================================
# CELL 2: Set your secrets
# ============================================================
# HOW TO ADD SECRETS IN KAGGLE:
#   1. Click the key icon (Add-ons) in the right panel
#   2. Or go to: Settings > Add-ons > Secrets
#   3. Add these 4 secrets (Label = value below, Value = your actual key):
#      - SUPABASE_URL
#      - SUPABASE_KEY
#      - NGROK_TOKEN
#      - HF_TOKEN
#   4. IMPORTANT: Toggle each secret to "Attached to notebook" (switch ON)
#   5. After adding secrets, you may need to RESTART the notebook session

import os

SUPABASE_URL = None
SUPABASE_KEY = None
NGROK_TOKEN  = None
HF_TOKEN     = None

# --- Method 1: Kaggle Secrets (recommended) ---
try:
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    SUPABASE_URL = secrets.get_secret('SUPABASE_URL')
    SUPABASE_KEY = secrets.get_secret('SUPABASE_KEY')
    NGROK_TOKEN  = secrets.get_secret('NGROK_TOKEN')
    HF_TOKEN     = secrets.get_secret('HF_TOKEN')
    print('Loaded secrets from Kaggle Secrets')
except ImportError:
    print('WARNING: kaggle_secrets not available (not running on Kaggle?)')
    print('         Will try environment variables or manual values below.')
except Exception as e:
    print(f'WARNING: Kaggle Secrets failed: {type(e).__name__}: {e}')
    print('         Make sure you added ALL 4 secrets AND toggled them ON.')
    print('         Go to: right panel > key icon > Add-ons > Secrets')

# --- Method 2: Environment variables (e.g., from .env or Kaggle env) ---
if not SUPABASE_URL:
    SUPABASE_URL = os.environ.get('SUPABASE_URL', '')
if not SUPABASE_KEY:
    SUPABASE_KEY = os.environ.get('SUPABASE_KEY', '')
if not NGROK_TOKEN:
    NGROK_TOKEN = os.environ.get('NGROK_TOKEN', '')
if not HF_TOKEN:
    HF_TOKEN = os.environ.get('HF_TOKEN', '')

# --- Method 3: Manual fallback (paste your values here if nothing else works) ---
# Uncomment and fill in ONLY if Kaggle Secrets and env vars don't work:
# SUPABASE_URL = 'https://xxxxx.supabase.co'
# SUPABASE_KEY = 'eyJhbGciOi...'
# NGROK_TOKEN  = '2xxx...'
# HF_TOKEN     = 'hf_xxx...'

# --- Set environment variables for the app ---
os.environ['SUPABASE_URL']      = SUPABASE_URL or ''
os.environ['SUPABASE_KEY']      = SUPABASE_KEY or ''
os.environ['PHI3_MODEL']        = 'microsoft/Phi-3-mini-4k-instruct'
os.environ['PHI3_QUANTIZATION'] = '4bit'
os.environ['T5_MODEL_PATH']     = 'espinajc/t5-auggregates-text2sql'
os.environ['ALLOWED_TABLES']    = 'ai_documents,Project,conversations'
os.environ['API_PORT']          = '8000'
os.environ['API_HOST']          = '0.0.0.0'
os.environ['ENVIRONMENT']       = 'production'
os.environ['CORS_ALLOW_ALL']    = 'true'

# --- Validate with helpful error messages ---
missing = []
if not SUPABASE_URL or SUPABASE_URL.startswith('YOUR') or SUPABASE_URL.startswith('https://xxxxx'):
    missing.append('SUPABASE_URL')
if not SUPABASE_KEY or SUPABASE_KEY.startswith('YOUR') or SUPABASE_KEY.startswith('eyJhbGciOi...'):
    missing.append('SUPABASE_KEY')
if not NGROK_TOKEN or NGROK_TOKEN.startswith('YOUR') or NGROK_TOKEN.startswith('2xxx'):
    missing.append('NGROK_TOKEN')
if not HF_TOKEN or HF_TOKEN.startswith('YOUR') or HF_TOKEN.startswith('hf_xxx'):
    missing.append('HF_TOKEN')

if missing:
    print()
    print('=' * 60)
    print(f'  MISSING SECRETS: {", ".join(missing)}')
    print('=' * 60)
    print()
    print('  To fix this in Kaggle:')
    print('  1. Click the KEY icon in the right panel (Add-ons > Secrets)')
    print('  2. Click "Add a new secret" for each missing one above')
    print('  3. Label = secret name exactly as shown (e.g. SUPABASE_URL)')
    print('  4. Value = your actual key/URL')
    print('  5. Toggle the switch to ATTACH it to this notebook')
    print('  6. After adding all secrets, RESTART the session:')
    print('     Run > Restart & Clear All Outputs > then re-run cells')
    print()
    print('  Or uncomment the manual values in Method 3 above and paste your keys.')
    raise ValueError(f'Missing secrets: {", ".join(missing)}. See instructions above.')

print(f'Supabase URL: {SUPABASE_URL[:40]}...')
print('All 4 secrets validated!')

In [ ]:
# ============================================================
# CELL 3: Clone your GitHub repo
# ============================================================
import os

GITHUB_REPO = 'https://github.com/espinajc2004-max/auggregates-ai-data-look-up.git'
REPO_DIR = '/kaggle/working/auggregates-ai-data-look-up'

if not os.path.exists(REPO_DIR):
    !git clone {GITHUB_REPO} {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull
    print(f'Updated existing repo at {REPO_DIR}')

os.chdir(REPO_DIR)
print(f'Working directory: {os.getcwd()}')

# Verify key files exist
for f in ['app/main.py', 'app/services/phi3_service.py', 'app/config/prompt_templates.py']:
    assert os.path.exists(f), f'Missing file: {f}'
print('All key files present!')

In [ ]:
# ============================================================
# CELL 4: Verify GPU
# ============================================================
import torch

print(f'PyTorch: {torch.__version__}')
print(f'CUDA:    {torch.cuda.is_available()}')

if torch.cuda.is_available():
    n_gpus = torch.cuda.device_count()
    print(f'GPUs visible: {n_gpus} (pinned to GPU 0)')
    name = torch.cuda.get_device_name(0)
    props = torch.cuda.get_device_properties(0)
    vram = props.total_memory / 1024**3
    print(f'GPU 0: {name} ({vram:.1f} GB VRAM)')
    print('\nGPU ready! Phi-3 4-bit needs ~2GB VRAM.')
else:
    print('No GPU! Go to Settings > Accelerator > GPU T4 x2')

In [ ]:
# ============================================================
# CELL 5: Login to HuggingFace
# ============================================================
from huggingface_hub import login
login(token=HF_TOKEN)
print('HuggingFace login successful!')

In [ ]:
# ============================================================
# CELL 6: Start FastAPI server + ngrok tunnel (IN-PROCESS)
# ============================================================
# Runs uvicorn INSIDE the notebook process to save ~3GB RAM.
# ============================================================
import threading, time, sys, os
import requests as req
import nest_asyncio
import uvicorn
from pyngrok import ngrok, conf

nest_asyncio.apply()

REPO_DIR = '/kaggle/working/auggregates-ai-data-look-up'
os.chdir(REPO_DIR)
if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

# Configure ngrok
conf.get_default().auth_token = NGROK_TOKEN

# Import the FastAPI app directly
from app.main import app

# Run uvicorn in a background thread (same process = shared GPU memory)
server_config = uvicorn.Config(
    app, host='0.0.0.0', port=8000,
    log_level='info', timeout_keep_alive=120
)
server = uvicorn.Server(server_config)

server_thread = threading.Thread(target=server.run, daemon=True)
server_thread.start()

print('Starting FastAPI server (in-process)...')
print('Models load in background. First run downloads ~2GB + ~770MB.')

# Wait for server to be ready
server_ready = False
for i in range(12):
    try:
        r = req.get('http://localhost:8000/api/health', timeout=3)
        if r.status_code == 200:
            server_ready = True
            break
    except:
        pass
    time.sleep(5)

if server_ready:
    print('Server is running on port 8000')
else:
    print('Server not responding yet, starting ngrok anyway...')

# Open ngrok tunnel
tunnel = ngrok.connect(8000)
public_url = tunnel.public_url

print()
print('=' * 60)
print(f'  SERVER IS LIVE!')
print(f'  Public URL:     {public_url}')
print(f'  Chat endpoint:  {public_url}/api/chat/hybrid')
print(f'  Health check:   {public_url}/api/health')
print('=' * 60)
print()
print('Copy the Public URL above and use it in your frontend!')
print()
print('Models are loading in background (~2-3 min).')
print('Run Cell 6B to monitor, or wait and run Cell 7 to test.')

In [ ]:
# ============================================================
# CELL 6B: Monitor download progress (run AFTER Cell 6)
# ============================================================
import os, time, requests as req

HF_CACHE = os.path.expanduser('~/.cache/huggingface/hub')
EXPECTED_MB = 2800  # ~2GB Phi-3 + ~770MB T5

def get_folder_size_mb(path):
    total = 0
    if not os.path.exists(path):
        return 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            try:
                total += os.path.getsize(fp)
            except OSError:
                pass
    return total / (1024 * 1024)

def get_gpu_info():
    try:
        import torch
        if torch.cuda.is_available():
            used = torch.cuda.memory_allocated() / 1024**3
            total = torch.cuda.get_device_properties(0).total_memory / 1024**3
            return f'{used:.1f}GB / {total:.1f}GB'
    except Exception:
        pass
    return 'N/A'

def check_model_status():
    try:
        r = req.get('http://localhost:8000/api/chat/hybrid/status', timeout=3)
        return r.json()
    except Exception:
        return None

print('Monitoring download + loading progress...')
print(f'Expected total: ~{EXPECTED_MB/1024:.1f} GB\n')

prev_size = 0
for i in range(60):
    size_mb = get_folder_size_mb(HF_CACHE)
    pct = min(100, (size_mb / EXPECTED_MB) * 100)
    speed = size_mb - prev_size
    speed_per_sec = speed / 5 if speed > 0 else 0
    gpu = get_gpu_info()
    status = check_model_status()
    
    bar = chr(9608) * int(pct // 5) + chr(9617) * (20 - int(pct // 5))
    print(f'  [{bar}] {pct:5.1f}% | {size_mb:.0f}/{EXPECTED_MB} MB | {speed_per_sec:.0f} MB/s | GPU: {gpu}')
    
    if status and status.get('phi3_loaded'):
        print(f'\nModels loaded! Pipeline: {status.get("pipeline")}')
        print('You can now run Cell 7 to test.')
        break
    
    if status and not status.get('loading_in_progress') and status.get('load_attempts', 0) >= 3:
        print(f'\nLoading failed after {status.get("load_attempts")} attempts.')
        print('Try: Restart kernel, then re-run all cells.')
        break
    
    prev_size = size_mb
    time.sleep(5)
else:
    print('\nStill loading after 5 min. Re-run this cell to keep monitoring.')

In [ ]:
# ============================================================
# CELL 7: Test the AI with a sample query
# ============================================================
import requests as req
import json

try:
    base_url = public_url
except NameError:
    base_url = 'http://localhost:8000'
    print(f'ngrok URL not available, using {base_url}\n')

# Check model loading status
print('Checking model status...')
try:
    status = req.get(f'{base_url}/api/chat/hybrid/status', timeout=10).json()
    print(f'   Phi-3 loaded: {status.get("phi3_loaded")}')
    print(f'   Loading in progress: {status.get("loading_in_progress")}')
    print(f'   Load attempts: {status.get("load_attempts")}/{status.get("max_attempts")}')
    print(f'   Pipeline: {status.get("pipeline")}\n')
except Exception as e:
    print(f'   Could not check status: {e}\n')

print('Sending test query: "show me the francis gays expenses file"')
print('(First query may be slow ~30-60s while models warm up)\n')

try:
    response = req.post(
        f'{base_url}/api/chat/hybrid',
        json={'query': 'show me the francis gays expenses file'},
        timeout=180
    )

    print(f'Status: {response.status_code}')
    data = response.json()
    print(f'Pipeline: {data.get("metadata", {}).get("pipeline", "unknown")}')
    print(f'SQL Source: {data.get("metadata", {}).get("sql_source", "n/a")}')
    print(f'Message: {data.get("message", "")}')
    print(f'Results: {data.get("metadata", {}).get("row_count", 0)} rows')
    
    if data.get('metadata', {}).get('pipeline') == 'phi3+t5':
        sql_src = data.get('metadata', {}).get('sql_source', '')
        print(f'\nFull AI pipeline is working! (Phi-3 + T5, SQL from: {sql_src})')
    elif data.get('metadata', {}).get('pipeline') == 'rule-based':
        print('\nUsing rule-based fallback (Phi-3 may still be loading)')
        print('Wait 1-2 minutes and try again.')
    
    print(f'\nFull response:\n{json.dumps(data, indent=2, ensure_ascii=False)}')

except req.exceptions.Timeout:
    print('Request timed out. Models may still be loading.')
    print('Wait 2 minutes and re-run this cell.')
except Exception as e:
    print(f'Error: {e}')
    print('Make sure Cell 6 completed successfully.')

In [ ]:
# ============================================================
# CELL 7B: Test Query Gating (out-of-scope + vague queries)
# ============================================================
import requests as req
import json

try:
    base_url = public_url
except NameError:
    base_url = 'http://localhost:8000'

test_cases = [
    {'label': 'Out-of-scope (weather)',   'query': 'what is the weather today?',       'expect_intent': 'out_of_scope'},
    {'label': 'Out-of-scope (recipe)',    'query': 'how do I cook adobo?',             'expect_intent': 'out_of_scope'},
    {'label': 'Vague (needs clarify)',    'query': 'show me data',                     'expect_intent': 'clarification'},
    {'label': 'Valid query',              'query': 'show all expenses for SJDM project', 'expect_intent': None},
]

print('Testing Query Gating Features\n')
passed = 0
for tc in test_cases:
    print(f'--- {tc["label"]} ---')
    print(f'  Query: "{tc["query"]}"')
    try:
        r = req.post(f'{base_url}/api/chat/hybrid', json={'query': tc['query']}, timeout=120)
        data = r.json()
        intent = data.get('intent', '')
        msg = data.get('message', '')[:120]
        pipeline = data.get('metadata', {}).get('pipeline', 'unknown')
        print(f'  Intent: {intent} | Pipeline: {pipeline}')
        print(f'  Message: {msg}')
        if tc['expect_intent'] and intent == tc['expect_intent']:
            print(f'  PASS (got expected intent: {tc["expect_intent"]})')
            passed += 1
        elif tc['expect_intent'] is None and intent not in ('out_of_scope', 'error'):
            print(f'  PASS (valid query processed normally)')
            passed += 1
        else:
            print(f'  Expected intent={tc["expect_intent"]}, got={intent}')
    except Exception as e:
        print(f'  Error: {e}')
    print()

print(f'Results: {passed}/{len(test_cases)} tests passed')
if passed == len(test_cases):
    print('Query gating is working correctly!')
else:
    print('Some tests did not match. Phi-3 is probabilistic, results may vary.')

In [ ]:
# ============================================================
# CELL 8: Health check + Model status + Debug
# ============================================================
import requests as req
import subprocess

try:
    base_url = public_url
except NameError:
    base_url = 'http://localhost:8000'

# Step 1: Check if server process is alive
print('--- Step 1: Server process check ---')
ps = subprocess.run(['pgrep', '-f', 'uvicorn'], capture_output=True, text=True)
if ps.stdout.strip():
    print(f'  Server PID(s): {ps.stdout.strip().replace(chr(10), ", ")}')
else:
    print('  SERVER IS DEAD - uvicorn process not found!')
    print('  Restart kernel and re-run all cells.')
    try:
        import torch
        if torch.cuda.is_available():
            used = torch.cuda.memory_allocated() / 1024**3
            total = torch.cuda.get_device_properties(0).total_memory / 1024**3
            print(f'  GPU VRAM: {used:.1f}GB / {total:.1f}GB')
    except Exception:
        pass

# Step 2: Health check
print('\n--- Step 2: Health endpoint ---')
try:
    r = req.get(f'{base_url}/api/health', timeout=10)
    if r.status_code == 200:
        print(f'  Health: {r.status_code} {r.json()}')
    else:
        print(f'  Health: {r.status_code} (raw: {r.text[:200]})')
except req.exceptions.ConnectionError:
    print('  Connection refused - server is not running.')
except Exception as e:
    print(f'  Error: {e}')

# Step 3: Model status
print('\n--- Step 3: Model status ---')
try:
    r = req.get(f'{base_url}/api/chat/hybrid/status', timeout=10)
    if r.status_code == 200:
        status = r.json()
        print(f'  Phi-3 loaded:       {status.get("phi3_loaded")}')
        print(f'  Loading in progress: {status.get("loading_in_progress")}')
        print(f'  Load attempts:      {status.get("load_attempts")}/{status.get("max_attempts")}')
        print(f'  Active pipeline:    {status.get("pipeline")}')
        if status.get('phi3_loaded'):
            print('\n  READY! Run Cell 7 to test.')
        elif status.get('loading_in_progress'):
            print('\n  Models still loading... wait and re-run this cell.')
        else:
            print(f'\n  Models not loaded after {status.get("load_attempts")} attempts.')
            print('  Try: Restart kernel, then re-run all cells.')
    else:
        print(f'  Status: {r.status_code} (raw: {r.text[:200]})')
except req.exceptions.ConnectionError:
    print('  Connection refused - server is not running.')
except Exception as e:
    print(f'  Error: {e}')

In [ ]:
# ============================================================
# CELL 9: Debug - Load Phi-3 directly (standalone, skip Cell 6)
# ============================================================
# WARNING: Do NOT run this if Cell 6 (server) is already running!
#          Two Phi-3 instances will cause CUDA OOM.
#          Use this ONLY for standalone debugging.
# ============================================================
import sys, os, traceback

REPO_DIR = '/kaggle/working/auggregates-ai-data-look-up'
os.chdir(REPO_DIR)
if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

import nest_asyncio
nest_asyncio.apply()

print('Attempting to load Phi-3+T5 directly...')
print('(This will take 3-5 min on first run as it downloads ~2GB)\n')

try:
    from app.services.phi3_service import Phi3Service
    svc = Phi3Service()
    print(f'Config: model={svc.config.model_name}')
    print(f'Config: quant={svc.config.quantization}')
    print(f'Config: device={svc.config.device}')
    print()
    svc._load_model()
    print('\n--- Model loaded! Testing inference ---')
    import asyncio
    result = asyncio.get_event_loop().run_until_complete(
        svc.process_query('show all expenses', 'test-user')
    )
    print(f'Pipeline response: {result.get("response", "")[:200]}')
    print(f'SQL: {result.get("sql", "")}')
    print(f'Rows: {result.get("row_count", 0)}')
    print('\nPhi-3+T5 is working!')
except Exception as e:
    print(f'Error: {type(e).__name__}: {e}')
    traceback.print_exc()

---
## Notes

**Kaggle vs Colab differences:**
- Kaggle gives 30 hours/week of GPU (vs Colab's ~90 min free sessions)
- Sessions can run up to 12 hours continuously
- Secrets are in Settings > Add-ons > Secrets (not sidebar key icon)
- Internet must be explicitly enabled in Settings

**Keeping the server alive:**
- Kaggle sessions stay alive longer than Colab
- Keep the browser tab open to prevent disconnection
- If disconnected, re-run all cells from Cell 1

**Connecting your frontend:**
- Use the ngrok Public URL from Cell 6 as your API base URL
- The URL changes every restart

**Performance:**
- First query after startup: ~30-60 seconds (model warmup)
- Subsequent queries: ~5-15 seconds
- T4 GPU has 15GB VRAM, Phi-3 4-bit uses ~2GB

**Troubleshooting:**
- `CUDA out of memory` > Restart kernel, re-run all cells
- `ngrok error` > Check your auth token
- `rule-based fallback` > Models still loading, wait 1-2 min